**> Web Scraping dados da Pnad Contínua no google colab**





In [ ]:

# Neste script eu utilizo o pacote 'PNADcIBGE' para baixar os microdados da PNAD Contínua e o
# pacote 'googledrive' para fazer o upload automático para o Google Drive.
#-----------------------------------------------------------------------------

# Instalando os pacotes necessários para o download e manipulação dos dados.
install.packages(c("PNADcIBGE", "googledrive", "dplyr", "httr", "readr"))

# Carregando as bibliotecas necessárias.
library(PNADcIBGE)
library(googledrive)
library(dplyr)

# --- PASSO 1: Autenticação com o Google Drive ---
#
# ETAPA DE LIMPEZA: Limpando qualquer autenticação anterior.
cat("Limpando credenciais de autenticação antigas...\n")
drive_deauth()

# ETAPA DE AUTENTICAÇÃO: Iniciando um novo processo de login.
# O argumento 'scopes' solicita permissão total (leitura e escrita).

cat("Iniciando nova autenticação com o Google Drive...\n")
drive_auth(email = "mauricio.meira@ufv.br", scopes = "https://www.googleapis.com/auth/drive")
cat("Autenticação bem-sucedida.\n")


# --- PASSO 2: Definição de Parâmetros e Pasta no Google Drive ---

anos_de_interesse <- 2013:2023
nome_pasta_drive <- "Pnad_Contínua_2013-2023"

# Procura pela pasta no Google Drive para obter o ID.
pasta_destino_info <- drive_find(n_max = 1, q = paste0("name = '", nome_pasta_drive, "' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"))

if (nrow(pasta_destino_info) == 0) {
  stop(paste0("ERRO: A pasta '", nome_pasta_drive, "' não foi encontrada. Verifique o nome ou crie-a no seu Google Drive."))
}

id_pasta_drive <- pasta_destino_info$id
cat(paste0("Pasta '", nome_pasta_drive, "' encontrada com sucesso no Google Drive.\n"))


# --- PASSO 3: Download Temporário e Upload para o Google Drive ---

for (ano in anos_de_interesse) {
  for (trimestre in 1:4) {

    temp_dir <- file.path(tempdir(), paste0("pnadc_", ano, "_", trimestre))
    if (!dir.exists(temp_dir)) dir.create(temp_dir, recursive = TRUE)

    tryCatch({

      cat("--------------------------------------------------\n")
      cat(paste0("Processando: ", ano, " - Trimestre ", trimestre, "\n"))

      get_pnadc(year = ano, quarter = trimestre, savedir = temp_dir)

      arquivos_para_upload <- list.files(temp_dir, full.names = TRUE, recursive = TRUE)

      if (length(arquivos_para_upload) > 0) {
        cat("Iniciando upload para o Google Drive...\n")
        for (arquivo in arquivos_para_upload) {
          drive_upload(media = arquivo,
                       path = as_id(id_pasta_drive),
                       name = basename(arquivo),
                       overwrite = TRUE)
        }
        cat("Upload concluído para ", ano, "T", trimestre, ".\n")
      }

    }, error = function(e) {
      cat(paste0("AVISO: Não foi possível processar ", ano, "T", trimestre, ". Motivo: ", e$message, "\n"))
    }, finally = {
      cat("Limpando arquivos temporários...\n")
      unlink(temp_dir, recursive = TRUE)
    })

    Sys.sleep(1)
  }
}

cat("--------------------------------------------------\n")
cat("PROCESSO DE COLETA DE DADOS E UPLOAD CONCLUÍDO.\n")
cat("Todos os arquivos foram salvos na pasta '", nome_pasta_drive, "' no seu Google Drive.\n")




Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




Limpando credenciais de autenticação antigas...
Iniciando nova autenticação com o Google Drive...


Is it OK to cache OAuth access credentials in the folder ~/.cache/gargle
between R sessions?
1: Yes
2: No
Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=1f3b5db4176a7854c301aba90fa59ceb&login_hint=mauricio.meira%40ufv.br&access_type=offline&prompt=consent



Autenticação bem-sucedida.
Pasta 'Pnad_Contínua_2013-2023' encontrada com sucesso no Google Drive.
--------------------------------------------------
Processando: 2013 - Trimestre 1


Paths of files downloaded in this function at the save directory provided are:

/tmp/RtmpfFwsEb/pnadc_2013_1/deflator_PNADC_2025_trimestral_010203.xls
/tmp/RtmpfFwsEb/pnadc_2013_1/dicionario_PNADC_microdados_trimestral.xls
/tmp/RtmpfFwsEb/pnadc_2013_1/input_PNADC_trimestral.sas
/tmp/RtmpfFwsEb/pnadc_2013_1/input_PNADC_trimestral.txt
/tmp/RtmpfFwsEb/pnadc_2013_1/PNADC_012013_20220916.zip
/tmp/RtmpfFwsEb/pnadc_2013_1/PNADC_012013.txt




...

Upload concluído para  2023 T 3 .
Limpando arquivos temporários...
--------------------------------------------------
Processando: 2023 - Trimestre 4


Paths of files downloaded in this function at the save directory provided are:

/tmp/RtmpfFwsEb/pnadc_2023_4/deflator_PNADC_2025_trimestral_010203.xls
/tmp/RtmpfFwsEb/pnadc_2023_4/dicionario_PNADC_microdados_trimestral.xls
/tmp/RtmpfFwsEb/pnadc_2023_4/input_PNADC_trimestral.sas
/tmp/RtmpfFwsEb/pnadc_2023_4/input_PNADC_trimestral.txt
/tmp/RtmpfFwsEb/pnadc_2023_4/PNADC_042023.txt
/tmp/RtmpfFwsEb/pnadc_2023_4/PNADC_042023.zip




Iniciando upload para o Google Drive...


Auto-refreshing stale OAuth token.

File trashed:

• deflator_PNADC_2025_trimestral_010203.xls
  <id: 1Kanhssbe6-sJHsDkSXySHNx4-JhEhsl6>

Local file:

• /tmp/RtmpfFwsEb/pnadc_2023_4/deflator_PNADC_2025_trimestral_010203.xls

Uploaded into Drive file:

• deflator_PNADC_2025_trimestral_010203.xls
  <id: 1N2cWSLgGvbP_LFQBLntFxIa1yjmtNV_0>

With MIME type:

• application/vnd.ms-excel

File trashed:

• Deflatores.zip <id: 1_5nGykV5HrKspHln4vZKrjqbHQd_A_4p>

Local file:

• /tmp/RtmpfFwsEb/pnadc_2023_4/Deflatores.zip

Uploaded into Drive file:

• Deflatores.zip <id: 1vyTD-qjCjp2IkYOgNcPSRCk8hum1YiIS>

With MIME type:

• application/zip

File trashed:

• Dicionario_e_input.zip <id: 1DOC2Q1E_zMGxn7M5RTm33p2O--5_iVxi>

Local file:

• /tmp/RtmpfFwsEb/pnadc_2023_4/Dicionario_e_input.zip

Uploaded into Drive file:

• Dicionario_e_input.zip <id: 1ZOWyG22Dk9-GK0nAA39w62G622kz922w>

With MIME type:

• application/zip

File trashed:

• dicionario_PNADC_microdados_trimestral.xls
  <id: 1wWtjP4iQc-iFDpTy

Upload concluído para  2023 T 4 .
Limpando arquivos temporários...
--------------------------------------------------
PROCESSO DE COLETA DE DADOS E UPLOAD CONCLUÍDO.
Todos os arquivos foram salvos na pasta ' Pnad_Contínua_2013-2023 ' no seu Google Drive.
